In [1]:
from util import load_user_data
import pandas as pd
import numpy as np
import sys
valid_users, tp_data, _ = load_user_data(filename="../data/xailabdata_all.csv")
valid_users_2, tp_data_2, _ = load_user_data(filename="../data/xailabdata_llm_agent.csv")

valid_users = valid_users | valid_users_2
tp_data.update(tp_data_2)

245 valid participants
{'control': 61, 'dashboard': 61, 'chatxai': 62, 'chatxaiboost': 61, 'chatxaiAuto': 0}
91 participants blindly rely on AI advice
{'control': 8, 'dashboard': 27, 'chatxai': 32, 'chatxaiboost': 24, 'chatxaiAuto': 0}
61 valid participants
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 61}
25 participants blindly rely on AI advice
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 25}


In [2]:
explainers = ["pdp", "shap", "whatIf", "decisionTree", "counterFactual"]
condition_usage = {}
for condition in ["control", "dashboard", "chatxai", "chatxaiboost", "chatxaiAuto"]:
    condition_usage[condition] = {}
    for explainer in explainers:
        condition_usage[condition][explainer] = []

limited_interaction = 0
limited_users = set()
for user in valid_users:
    # print(tp_data[user].keys())
    tp_condition = tp_data[user]["condition"]
    if tp_condition in ["control", "dashboard"]:
        continue
    # print(tp_condition)
    # print(tp_data[user].keys())
    task_order = tp_data[user]["task_order"]
    xai_usage = tp_data[user]["xai_usage"]
    tp_condition_usage = {}
    for explainer in explainers:
        tp_condition_usage[explainer] = 0
    for task_id in task_order:
        # print(xai_usage)
        if task_id not in xai_usage:
            print(xai_usage, task_id)
            sys.exit(-1)
        metadata = xai_usage[task_id]["explainer_counter"]
        total_usage_tp = 0
        # print(tp_condition, metadata)
        for explainer in explainers:
            assert isinstance(metadata[explainer], int)
            # condition_usage[tp_condition][explainer].append(metadata[explainer])
            tp_condition_usage[explainer] += metadata[explainer]
            total_usage_tp += metadata[explainer]
        if total_usage_tp <= 2:
            limited_interaction += 1
            limited_users.add(user)
    for explainer in explainers:
        condition_usage[tp_condition][explainer].append(tp_condition_usage[explainer])
print(limited_interaction)
print(len(limited_users))
# sys.exit(-1)

778
157


In [3]:
from scipy.stats import kruskal, mannwhitneyu
def post_hoc_comparison(data_list_1, data_list_2, name1, name2):
	print("Use pots-hoc analysis")
	threshold = 0.05 / 4
	flag = False
	statistic, pvalue = mannwhitneyu(data_list_1, data_list_2, alternative='greater')
	if pvalue < threshold:
		print("Alternative {} > {},".format(name1, name2), "pvalue %.4f"%pvalue, "statistic %.4f"%statistic)
		flag = True
	statistic, pvalue = mannwhitneyu(data_list_1, data_list_2, alternative='less')
	if pvalue < threshold:
		print("Alternative {} < {},".format(name1, name2), "pvalue %.4f"%pvalue, "statistic %.4f"%statistic)
		flag = True
	if not flag:
		print("No significant difference with post-hoc analysis")

for explainer in explainers:
    data_list_1 = condition_usage["chatxai"][explainer]
    data_list_2 = condition_usage["chatxaiboost"][explainer]
    data_list_3 = condition_usage["chatxaiAuto"][explainer]
    statistic, pvalue = kruskal(data_list_1, data_list_2, data_list_3)
    print(explainer)
    if pvalue < 0.05 / 4:
        post_hoc_comparison(data_list_1, data_list_2, "CXAI", "PCXAI")
        post_hoc_comparison(data_list_1, data_list_3, "CXAI", "LLM Agent")
        post_hoc_comparison(data_list_2, data_list_3, "PCXAI", "LLM Agent")
    print(len(data_list_1), len(data_list_2), len(data_list_3))
    print("kruskal test result: H:{:.2f}, p:{:.3f}".format(statistic, pvalue))
    print("On average: M(CXAI):{:.1f} M(PCXAI):{:.1f} M(LLM Agent):{:.1f}".format(np.mean(data_list_1), np.mean(data_list_2), np.mean(data_list_3)))
    print("SD(CXAI):{:.1f} SD(PCXAI):{:.1f} SD(LLM Agent):{:.1f}".format(np.std(data_list_1), np.std(data_list_2), np.std(data_list_3)))
    print("-" * 17)

pdp
Use pots-hoc analysis
No significant difference with post-hoc analysis
Use pots-hoc analysis
Alternative CXAI > LLM Agent, pvalue 0.0000 statistic 3442.0000
Use pots-hoc analysis
Alternative PCXAI > LLM Agent, pvalue 0.0000 statistic 3287.5000
62 61 61
kruskal test result: H:76.85, p:0.000
On average: M(CXAI):13.5 M(PCXAI):14.1 M(LLM Agent):3.6
SD(CXAI):7.3 SD(PCXAI):8.5 SD(LLM Agent):3.5
-----------------
shap
Use pots-hoc analysis
No significant difference with post-hoc analysis
Use pots-hoc analysis
Alternative CXAI < LLM Agent, pvalue 0.0001 statistic 1169.5000
Use pots-hoc analysis
Alternative PCXAI < LLM Agent, pvalue 0.0059 statistic 1373.5000
62 61 61
kruskal test result: H:15.32, p:0.000
On average: M(CXAI):6.7 M(PCXAI):7.8 M(LLM Agent):9.5
SD(CXAI):3.7 SD(PCXAI):3.3 SD(LLM Agent):4.3
-----------------
whatIf
Use pots-hoc analysis
No significant difference with post-hoc analysis
Use pots-hoc analysis
Alternative CXAI < LLM Agent, pvalue 0.0009 statistic 1274.5000
Use pots-

In [4]:
condition_FS = {}
for condition in ["dashboard", "chatxai", "chatxaiboost", "chatxaiAuto"]:
    condition_FS[condition] = []
for user in valid_users:
    condition = tp_data[user]["condition"]
    if condition == "control":
        continue
    feature_switch = tp_data[user]["explanation_understanding"]["feature_switch"]
    condition_FS[condition].append(feature_switch)
for condition in ["dashboard", "chatxai", "chatxaiboost", "chatxaiAuto"]:
    print("{}: M: {:.2f}".format(condition, np.mean(condition_FS[condition])))

data_list_1 = condition_FS["chatxai"]
data_list_2 = condition_FS["chatxaiboost"]
data_list_3 = condition_FS["chatxaiAuto"]
statistic, pvalue = kruskal(data_list_1, data_list_2, data_list_3)
print("kruskal test result: H:{:.2f}, p:{:.3f}".format(statistic, pvalue))
print("On average: M(CXAI):{:.2f} M(PCXAI):{:.2f} M(LLM Agent):{:.2f}".format(np.mean(data_list_1), np.mean(data_list_2), np.mean(data_list_3)))
print("-" * 17)

dashboard: M: 0.96
chatxai: M: 0.93
chatxaiboost: M: 0.98
chatxaiAuto: M: 0.97
kruskal test result: H:0.62, p:0.733
On average: M(CXAI):0.93 M(PCXAI):0.98 M(LLM Agent):0.97
-----------------
